The enum keyword is used to declare an enumeration, a distinct type that consists of a set of named constants called the enumerator list. 

In [1]:
from enum import Enum

Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [6]:
for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12


In [5]:
Month.__members__.items()

odict_items([('Jan', <Month.Jan: 1>), ('Feb', <Month.Feb: 2>), ('Mar', <Month.Mar: 3>), ('Apr', <Month.Apr: 4>), ('May', <Month.May: 5>), ('Jun', <Month.Jun: 6>), ('Jul', <Month.Jul: 7>), ('Aug', <Month.Aug: 8>), ('Sep', <Month.Sep: 9>), ('Oct', <Month.Oct: 10>), ('Nov', <Month.Nov: 11>), ('Dec', <Month.Dec: 12>)])

In [7]:
class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, Gender):
        self.name = name
        self.gender = Gender

In [10]:
Gender.Female

<Gender.Female: 1>

In [20]:
alice = Student("Alice", Gender.Female)

In [21]:
alice.name

'Alice'

In [22]:
alice.gender

<Gender.Female: 1>

The usage of Python @property.

In [63]:
class User(object):
    
    @property
    def password(self):
        # Use to get the attribute
        self._password = 'm'.join(self._password.split("s"))
        return self._password
    
    @password.setter
    def password(self, value):
        # Use to set the attribute
        if len(value) <= 8:
            raise ValueError('Password length too short.')
        self._password = value

In [64]:
user = User() # new instance of user object

In [65]:
user.password = "badpass"

ValueError: Password length too short.

In [66]:
user.password = "goodpassword"

In [67]:
user.password

'goodpammword'

In [68]:
abs(-11)

11

In [80]:
def fn(self, name):
    return name

In [81]:
User = type('User', (object,), dict(get_name=fn))

In [82]:
user = User()
user.get_name('my name')

'my name'

`Select` has result from the db, but `Insert, Update, Delete` will return the interation of row.

In [89]:
import sqlite3
conn = sqlite3.connect('file:/tmp/data.db', uri=True)

In [90]:
cur = conn.cursor()

In [98]:
cur = cur.execute('SELECT * FROM todo_list;')

In [99]:
cur.rowcount

-1

In [100]:
cur.fetchall()

[(1, 'hello', 0)]

In [96]:
cur.rowcount

-1

Define what is the User object should be like.

In [118]:
class User(Model):
    __table__ = 'users'
    
    id = IntegerField(primary_key=True)
    name = StringField()
    # id, name, __table__ is will be the attribute of the class not for the class instance.
    # The difference of that is will can call the the attribute direactly user User.id. no more need instance.

NameError: name 'Model' is not defined

In [115]:
# create instance
user = User(id=123, name='Alice')
# save to the DB
user.insert()
# query the user object
users = User.findAll()

In [219]:
# Define the basic class of ORM Model
class Model(dict):
    def __init__(self, **kwargs):
        super(Model, self).__init__(**kwargs)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError("'Model' object not has attribute {}".format(key))
    
    def __setattr__(self, key, value):

        self[key] = value

In [220]:
# Model class inherited dict, the instance of Model is the dict.
user_dict = Model(id=123, name='Alice')

In [221]:
user_dict.age
user_dict

AttributeError: 'Model' object not has attribute age

In [209]:
def fn(name):
    print
User = type('User', (object,), dict(func = lambda self, n: print(n)))

In [211]:
user = User()
user.func("hello")
user.a

hello


AttributeError: 'User' object has no attribute 'a'

In [1]:
class Field(object):
    def __init__(self, column_type, primary_key, default):
        self.column_type = column_type
        self.primary_key = primary_key
        self.default = default
    def __repr__(self):
        return '<{}, {}>'.format(self.__class__.__name__, self.column_type)
    
class TextField(Field):
    
    def __init__(self, column_type='TEXT NOT NULL', default="", primary_key=False):
        super(TextField, self).__init__(column_type, primary_key, default)
        
class IntegerField(Field):
    
    def __init__(self, column_type='INTEGER NOT NULL', default=1, primary_key=False):
        super(IntegerField, self).__init__(column_type, primary_key, default)
        
class BooleanField(Field):
    def __init__(self, column_type='BOOLEAN NOT NULL', default=False):
        super(BooleanField, self).__init__(column_type, False, default)

In [282]:
class ModelMetaclass(type):
    """
    Meta class of Model.
    """
    def __new__(cls, name, bases, attrs):
        """
        Customize the class instance creation

        Every child class which inherit from the `Model` class will automatically
        create the following attributes of that class.

        Definition Class Attributes:
        ----------------------------
            PRIMARY_KEY : Store the attribute name which is belong to
                        `Field` object with set argument (primary_key = True)
            MAPPINGS : A dict which storing the relationship of the class
                       attribute name and it's bounded `Field` object. 
            TABLE_NAME : The name of table which be took from the class name.
            FIELDS : A list which store the attribute name which is belong
                     to `Field` object without set argument (primary_key =True)

        Parametes:
        ----------
        cls : object
            Object of the class.
        name : str
            Name of the class.
        bases : tuple
            A tuple of class that be inherited from.
        attrs : dict
            A dict of class attributes
        
        Returns
        -------
            A new instance of class
        """
        if name == 'Model':
            return type.__new__(cls, name, bases, attrs)
        table_name = name
        primary_key = None
        fields = []
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                mappings[k] = v
                if v.primary_key:
                    if primary_key:
                        raise NameError('Primary key should only be one.')
                    else:
                        primary_key = k
                else:
                    fields.append(k)
        if primary_key is None:
            raise NameError('Primary key not found.')
        # Delete the class attributes which is belong to `Field` object.
        # Because the class attributes may be overide by the same name of 
        # the class instance attributes.
        for key in mappings:
            print(key)
            attrs.pop(key)

        attrs['PRIMARY_KEY'] = primary_key
        attrs['MAPPINGS'] = mappings
        attrs['TABLE_NAME'] = table_name
        attrs['FIELDS'] = fields
        return type.__new__(cls, name, bases, attrs)
    
class Model(dict, metaclass=ModelMetaclass):
    """
    The base class of DB model that provides simple OR mapping.
    """
    def __init__(self, **kwargs):
        """
        Inherit from dict object.
        """
        super(Model, self).__init__(**kwargs)

    def __getattr__(self, key):
        """
        Get instance attributes method
        
        Usage : 
            model = Model(id=1)
            model.id //will return 1
        """
        try:
            return self[key]
        except KeyError:
            raise AttributeError("'Model' object not has attribute {}".format(key))

    def __setattr__(self, key, value):
        """
        Set instance attributes method

        Usage : 
            model = Model()
            model.id = 1
            model // will return {'id' : 1}
        """
        self[key] = value

    def _get_value_or_default(self, key):
        """
        Get the instance attribute value
        
        Parameters
        ----------
        key : str
            The name of attribute
        
        Returns
        -------
        value : str | int | bool
            The value of the instance attribute.
            Or the default value which set in the 
            `Field` Object.
        """
        value = getattr(self, key, None)
        if value is None:
            field = self.MAPPINGS[key]
            if field.default is not None:
                value = field.default
                setattr(self, key, value)
        return value

    @classmethod
    def _select(cls):
        """
        SELECT SQL statement
        """
        return 'SELECT {}, {} FROM {}'.format(
            cls.PRIMARY_KEY,
            ', '.join(cls.FIELDS),
            cls.TABLE_NAME
        )

    @classmethod
    def _update(cls):
        """
        UPDATE SQL statement
        """
        return 'UPDATE {} SET  {} where {}=?'.format(
            cls.TABLE_NAME,
            ', '.join(map(lambda f: '{}=?'.format(f), cls.FIELDS)),
            cls.PRIMARY_KEY
        )

    @classmethod
    def _delete(cls):
        """
        DELETE SQL statement
        """
        return 'DELETE FROM {} WHERE {}=?'.format(
            cls.TABLE_NAME,
            cls.PRIMARY_KEY
        )

    @classmethod
    def create_table(cls):
        """
        Execute create table SQL statement.
        """
        values = []
        for key, field in cls.MAPPINGS.items():
            sql = ' '.join(
                [key, field.column_type, 'PRIMARY KEY' if field.primary_key else ''])
            values.append(sql)
        #cur.execute(sql, args)
        return 'CREATE TABLE IF NOT EXISTS {} ({})'.format(cls.TABLE_NAME, ','.join(values))

    def to_dict(self, result):
        # result is tuple
        pass
        
    
    @classmethod
    def drop_table(cls):
        """
        Execute drop table SQL statement.
        """
        return 'DROP TABLE {}'.format(cls.TABLE_NAME)
        
    @classmethod
    def find_all(cls, condition=None,  **kwargs):
        """
        If want find item base on the primary_key the args just need the values.
        """
        sql = [cls._select()]
        args = []
        if condition:
            for key, value in condition.items():
                sql.append('WHERE {}=?'.format(key))
                args = value
        order_by = kwargs.get('order_by')
        if order_by:
            sql.append('ORDER BY')
            sql.append(order_by)
        #cur.execute(sql, args)
        [cls(*r) for r in result]
        return ' '.join(sql), args

    @classmethod
    def find(cls, primary_key):
        sql = '{} WHERE {} = ?'.format(cls._select(), cls.PRIMARY_KEY)
        #cur.execute(sql, [prima_key])
        [cls(*r) for r in result]
        return sql, [primary_key]

    def remove(self):
        #cur.execute(self._delete(), [self._get_value_or_default(self.PRIMARY_KEY)])
        pass

    def fn(self, k):
        if k in self.FIELDS:
            return k

    def update(self):
        return 'UPDATE {} SET  {} where {}=?'.format(
            self.TABLE_NAME,
            ', '.join(map(lambda f: '{}=?'.format(f), self)),
            self.PRIMARY_KEY
        )
        #cur.execute(self._update(), [args])


class Todo(Model):
    id = IntegerField(primary_key=True)
    context = TextField()
    flag = BooleanField()


id
context
flag


In [285]:
Todo.MAPPINGS['id'].default

1

In [270]:
sql = Todo(id=1, flag=True).update()
sql

'UPDATE Todo SET  id=?, flag=? where id=?'

In [6]:
import sqlite3

In [15]:
def execute_return_rowcount(sql, args=[], autocommit=True):
    conn = sqlite3.connect('data.sql')
    cursor = conn.cursor()
    cursor.execute(sql, args)
    affected = cursor.rowcount
    if autocommit:
        conn.commit()
    return affected

In [14]:
def execute_return_result(sql, args=[], size=None):
    conn = sqlite3.connect('data.sql')
    cursor = conn.cursor()
    cursor.execute(sql, args)
    if size:
        result = cursor.fetchmany(size)
    else:
        result = cursor.fetchall()
    return self.to_dict(result)

In [274]:
Todo.id

AttributeError: type object 'Todo' has no attribute 'id'

In [97]:
Todo.TABLE_NAME

'Todo'

In [102]:
sql ="PRAGMA table_info(Todo)";

In [103]:
execute_return_rowcount(sql)

-1

In [143]:
sql, args = Todo.find_all({'id': 10})
sql, args

('SELECT id, context, flag FROM Todo WHERE id=?', 10)

In [144]:
result = execute_return_result(sql, [args])
result

[]

In [26]:
class foo(dict):
    def __init__(self,id, contect, flsg **kwargs):
        super(foo, self).__init(**kwargs)

[(3, 'hello', 0), (2, 'hello', 0), (1, 'hello', 0)]

In [28]:
def foo(*r):
    print(r)

In [71]:
todo_list = [cls(*r) for r in result]

In [73]:
for todo in todo_list:
    print(todo.id)

3
2
1


In [106]:
list(list(map(lambda f: f, [1, 2])))

[1, 2]

In [148]:
keys = {"c": 2, "b": 3}

In [149]:
values = []
m = zip(keys, values)

In [154]:
bar = dict(list(m))
bar

{}

In [173]:
values = [1, 2]
1 in values

True

In [156]:
foo(bar)

TypeError: foo() takes 0 positional arguments but 1 was given

In [163]:
0 is None

False

In [273]:
import time
t1 = time.time
time.sleep(2)
t2 = time.time
t1-t2

TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'builtin_function_or_method'